In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read and filter out irrelevant entries
df = pd.read_csv('./input/input.csv')
df = df[(df['State'] != 'Refused') & (df['State'] != 'Reminder sent') & (df['State'] != 'excluded') ]

# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")

### Histograms

In [ ]:
# Uniformizing values
df['Company size'][df['Company size'] == '51-100'] = '51-500'
df['Company size'][df['Company size'] == '100-500'] = '51-500'
df['Company size'][df['Company size'] == '5000-10000'] = '500-10000'
# Sorting bins
df['Company size'] = pd.Categorical(df['Company size'], ['1-10','11-50','51-500','500-10000','10001+'])


# Create the plot
ax = sns.histplot(data=df, x="Company size")
# ax.set_title('')
ax.set_ylabel('# interviewees')


plt.show()

In [ ]:
# Sorting bins
df['Experience (general)'] = pd.Categorical(df['Experience (general)'], ['0-2', '3-5', '6-9', '10+'])


# Create the plot
ax = sns.histplot(data=df, x="Experience (general)")
# ax.set_title('')
ax.set_ylabel('# interviewees')
ax.set_ylim(0,12)


plt.show()

In [ ]:
# Sorting bins
df['Experience (ML)'] = pd.Categorical(df['Experience (ML)'], ['0-2', '3-5', '6-9', '10+'])


# Create the plot
ax = sns.histplot(data=df, x="Experience (ML)")
# ax.set_title('')
ax.set_ylabel('# interviewees')
ax.set_ylim(0,15)



plt.show()

### Pie charts